In [7]:
from model.BSRModel import BSRModel


In [8]:
league =  'BZ-Standard'
season_start = 2026
season_end = 2046
pa_limit = 10
epochs = 10000
ratings_type = "Standard"

In [9]:
bsrModel = BSRModel(league, season_start, season_end, ratings_type, "10")
bsrModel.load_data(pa_limit)

epoch_count, loss = bsrModel.train(epochs)
print(f"Epoch [{epoch_count}/{epochs}], Loss: {loss:.7f}")

test_loss = bsrModel.evaluate()
print(f"Test Error: {test_loss:.7f}")

feature_importance = bsrModel.feature_importance()
print(feature_importance)

bsrModel.save_model()

predictions = bsrModel.predict(season_end+1, pa_limit)
predictions.sort_values(by='Predictions', ascending=False, inplace=True)
print(predictions.head(100))

Empty DataFrame
Columns: [SPE, SR, STE, RUN, WT, BSR600]
Index: []
(9066, 6)
Epoch [2000/10000], Best Loss: 1.9513831
Epoch [4000/10000], Best Loss: 1.9272705
Epoch [6000/10000], Best Loss: 1.9147773
Epoch [8000/10000], Best Loss: 1.9072263
Epoch [10000/10000], Best Loss: 1.9010273
Epoch [9999/10000], Loss: 1.9019507
Test Error: 2.0345027
   mean_abs_shap  stdev_abs_shap name
2       1.269252        1.384400  STE
3       0.626504        0.490116  RUN
0       0.386622        0.280475  SPE
4       0.142062        0.135634   WT
1       0.122968        0.182917   SR
     SPE  SR  STE  RUN   WT  Predictions     ID
269   75  80   80   80  186    13.135908  36772
272   70  80   80   80  192    13.009398  36801
193   70  80   80   80  202    12.879237  34070
209   80  80   80   75  192    12.818799  34386
420   65  80   80   80  193    12.593886  47523
..   ...  ..  ...  ...  ...          ...    ...
1     80  65   55   65  205     1.294592  19760
3     45  50   55   75  195     1.249964  22152